# Démonstration : ECDSA (Elliptic Curve Digital Signature Algorithm)

**Objectifs** :
- Comprendre les courbes elliptiques en cryptographie
- Implémenter ECDSA (signatures sur courbes elliptiques)
- Comparer avec RSA et DSA
- Utiliser Ed25519 (courbe moderne)

In [ ]:
from cryptography.hazmat.primitives.asymmetric import ec, ed25519
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.backends import default_backend
import secrets
import time

## 1. Courbes Elliptiques : Introduction

**Équation** : $y^2 = x^3 + ax + b \pmod{p}$

**Opération de groupe** : Addition de points
- $P + Q = R$ (géométriquement : ligne passant par P et Q)
- $nP = P + P + \ldots + P$ (n fois)

**Problème difficile** : ECDLP (Elliptic Curve Discrete Logarithm Problem)
- Donné $P$ et $Q = nP$, trouver $n$ est difficile

**Avantage** :
- Clés courtes (256 bits) = Sécurité RSA-3072
- Calculs plus rapides
- Moins de bande passante

## 2. ECDSA : Signatures sur Courbes Elliptiques

**Setup** :
- Courbe $E$, point générateur $G$ d'ordre $n$
- Clé privée : $d \xleftarrow{\$} [1, n-1]$
- Clé publique : $Q = dG$

**Signature de $m$** :
1. $k \xleftarrow{\$} [1, n-1]$ (nonce aléatoire, crucial !)
2. $(x, y) = kG$
3. $r = x \mod n$
4. $s = k^{-1}(H(m) + dr) \mod n$
5. Signature : $(r, s)$

**Vérification** :
1. $u_1 = H(m) \cdot s^{-1} \mod n$
2. $u_2 = r \cdot s^{-1} \mod n$
3. $(x, y) = u_1 G + u_2 Q$
4. Valide si $r \equiv x \pmod{n}$

## 3. ECDSA avec secp256r1 (NIST P-256)

In [ ]:
def ecdsa_demo():
    """
    Démonstration complète d'ECDSA avec secp256r1.
    """
    print("=" * 70)
    print("ECDSA avec secp256r1 (NIST P-256)")
    print("=" * 70)
    
    # Génération de clés
    print("\n👤 Génération de clés ECDSA :")
    start = time.perf_counter()
    private_key = ec.generate_private_key(ec.SECP256R1(), default_backend())
    elapsed = time.perf_counter() - start
    
    public_key = private_key.public_key()
    
    print(f"   Temps de génération : {elapsed*1000:.2f} ms")
    print(f"   Courbe : secp256r1 (NIST P-256)")
    print(f"   Taille clé privée : 256 bits")
    print(f"   Taille clé publique : 2 × 256 bits (point (x, y))")
    
    # Sérialisation des clés
    private_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )
    
    public_pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    
    print(f"\n🔑 Clés (format PEM) :")
    print(f"   Clé privée : {len(private_pem)} bytes")
    print(f"   Clé publique : {len(public_pem)} bytes")
    
    # Message à signer
    message = b"Transaction: Alice sends 10 BTC to Bob"
    
    print(f"\n📝 Message à signer : {message}")
    
    # Signature
    print(f"\n✍️  Signature ECDSA :")
    start = time.perf_counter()
    signature = private_key.sign(
        message,
        ec.ECDSA(hashes.SHA256())
    )
    time_sign = time.perf_counter() - start
    
    print(f"   Temps : {time_sign*1000:.2f} ms")
    print(f"   Signature (hex) : {signature.hex()[:64]}...")
    print(f"   Taille : {len(signature)} bytes")
    
    # Vérification
    print(f"\n✅ Vérification de la signature :")
    start = time.perf_counter()
    try:
        public_key.verify(
            signature,
            message,
            ec.ECDSA(hashes.SHA256())
        )
        time_verify = time.perf_counter() - start
        print(f"   Temps : {time_verify*1000:.2f} ms")
        print(f"   Résultat : VALIDE ✅")
        print(f"   → Signature authentique (vient de la clé privée)")
        print(f"   → Message intègre (pas modifié)")
    except Exception as e:
        print(f"   Résultat : INVALIDE ❌")
        print(f"   Erreur : {e}")
    
    return private_key, public_key, signature

private_key, public_key, signature = ecdsa_demo()

## 4. Tests de Robustesse

In [ ]:
def ecdsa_robustness_tests():
    """
    Tests de robustesse d'ECDSA.
    """
    print("\n" + "=" * 70)
    print("TESTS DE ROBUSTESSE ECDSA")
    print("=" * 70)
    
    message = b"Original message"
    
    # Test 1 : Modification du message
    print(f"\n🔍 TEST 1 : Modification du message")
    modified_message = b"Modified message"
    
    print(f"   Message original : {message}")
    print(f"   Message modifié  : {modified_message}")
    
    try:
        public_key.verify(
            signature,
            modified_message,
            ec.ECDSA(hashes.SHA256())
        )
        print(f"   ❌ ERREUR : Signature valide sur message modifié !")
    except Exception:
        print(f"   ✅ Signature INVALIDE (attendu)")
        print(f"   → Modification DÉTECTÉE")
    
    # Test 2 : Corruption de la signature
    print(f"\n🔍 TEST 2 : Corruption de la signature")
    corrupted_signature = bytearray(signature)
    corrupted_signature[0] ^= 0xFF
    
    try:
        public_key.verify(
            bytes(corrupted_signature),
            message,
            ec.ECDSA(hashes.SHA256())
        )
        print(f"   ❌ ERREUR : Signature corrompue acceptée !")
    except Exception:
        print(f"   ✅ Signature INVALIDE (attendu)")
        print(f"   → Corruption DÉTECTÉE")
    
    # Test 3 : Mauvaise clé publique
    print(f"\n🔍 TEST 3 : Vérification avec mauvaise clé publique")
    wrong_private_key = ec.generate_private_key(ec.SECP256R1(), default_backend())
    wrong_public_key = wrong_private_key.public_key()
    
    try:
        wrong_public_key.verify(
            signature,
            message,
            ec.ECDSA(hashes.SHA256())
        )
        print(f"   ❌ ERREUR : Signature valide avec mauvaise clé !")
    except Exception:
        print(f"   ✅ Signature INVALIDE (attendu)")
        print(f"   → Authentification échouée (mauvais signataire)")
    
    # Test 4 : Randomisation (signatures différentes)
    print(f"\n🔍 TEST 4 : Randomisation des signatures")
    sig1 = private_key.sign(message, ec.ECDSA(hashes.SHA256()))
    sig2 = private_key.sign(message, ec.ECDSA(hashes.SHA256()))
    
    print(f"   Signature 1 : {sig1.hex()[:32]}...")
    print(f"   Signature 2 : {sig2.hex()[:32]}...")
    print(f"   Différentes : {sig1 != sig2} ✅")
    print(f"   → Nonce k aléatoire (sécurité)")
    
    # Vérifier que les deux sont valides
    try:
        public_key.verify(sig1, message, ec.ECDSA(hashes.SHA256()))
        public_key.verify(sig2, message, ec.ECDSA(hashes.SHA256()))
        print(f"   Les deux signatures sont valides ✅")
    except:
        print(f"   ❌ ERREUR : Une signature invalide !")

ecdsa_robustness_tests()

## 5. Attaque : Nonce Reuse (k réutilisé)

**CATASTROPHE** : Si le même $k$ est utilisé pour deux signatures, la clé privée peut être récupérée !

**Exemple célèbre** : PlayStation 3 hack (2010)
- Sony a réutilisé le même $k$ pour toutes les signatures
- Clé privée récupérée → Jeux piratés signés comme légitimes

In [ ]:
def ecdsa_nonce_reuse_attack_theory():
    """
    Explication théorique de l'attaque par réutilisation de nonce.
    """
    print("\n" + "=" * 70)
    print("ATTAQUE : Nonce Reuse (k réutilisé)")
    print("=" * 70)
    
    print(f"\n⚠️  Scénario vulnérable :")
    print(f"   Deux signatures (r₁, s₁) et (r₂, s₂) avec le MÊME k")
    
    print(f"\n💡 Rappel ECDSA :")
    print(f"   s = k⁻¹(H(m) + dr) mod n")
    print(f"   r = x-coordinate of kG")
    
    print(f"\n🔍 Si k réutilisé pour m₁ et m₂ :")
    print(f"   s₁ = k⁻¹(H(m₁) + dr)")
    print(f"   s₂ = k⁻¹(H(m₂) + dr)")
    print(f"   r₁ = r₂ = r (même k → même r !)")
    
    print(f"\n💥 Attaque :")
    print(f"   s₁ - s₂ = k⁻¹(H(m₁) - H(m₂))")
    print(f"   → k = (H(m₁) - H(m₂)) / (s₁ - s₂) mod n")
    print(f"   → d = (s₁·k - H(m₁)) / r mod n")
    print(f"\n   CLÉ PRIVÉE RÉCUPÉRÉE ! 💀")
    
    print(f"\n📜 Exemples historiques :")
    print(f"   - PlayStation 3 (2010) : k constant → hack complet")
    print(f"   - Bitcoin (2013) : Wallet Android, k faible → vol de BTC")
    print(f"   - Blockchain.info (2014) : k prédictible → vol")
    
    print(f"\n✅ PROTECTIONS :")
    print(f"   1. k DOIT être aléatoire cryptographiquement sûr")
    print(f"   2. k DOIT être unique pour chaque signature")
    print(f"   3. Utiliser RFC 6979 : k déterministe = HMAC(d, m)")
    print(f"      → k unique par message, pas de hasard requis")
    print(f"   4. Utiliser Ed25519 : k = H(secret || m) (intégré)")

ecdsa_nonce_reuse_attack_theory()

## 6. Ed25519 : Courbe Moderne (Recommandée)

**Avantages sur ECDSA** :
- ✅ Plus rapide (courbe Curve25519 optimisée)
- ✅ Résistant aux side-channels (constant-time)
- ✅ Nonce déterministe (pas de risque de réutilisation)
- ✅ Signatures plus courtes (64 bytes)
- ✅ Clés plus courtes (32 bytes)

**Utilisé dans** :
- SSH (ssh-ed25519)
- Signal Protocol
- Tor
- Cryptomonnaies (Monero, Stellar, etc.)

In [ ]:
def ed25519_demo():
    """
    Démonstration d'Ed25519 (courbe moderne).
    """
    print("\n" + "=" * 70)
    print("ED25519 : Courbe Moderne (Recommandée)")
    print("=" * 70)
    
    # Génération de clés
    print("\n👤 Génération de clés Ed25519 :")
    start = time.perf_counter()
    private_key = ed25519.Ed25519PrivateKey.generate()
    elapsed = time.perf_counter() - start
    
    public_key = private_key.public_key()
    
    print(f"   Temps de génération : {elapsed*1000:.2f} ms")
    print(f"   Courbe : Curve25519 (Edwards form)")
    
    # Sérialisation
    private_bytes = private_key.private_bytes(
        encoding=serialization.Encoding.Raw,
        format=serialization.PrivateFormat.Raw,
        encryption_algorithm=serialization.NoEncryption()
    )
    
    public_bytes = public_key.public_bytes(
        encoding=serialization.Encoding.Raw,
        format=serialization.PublicFormat.Raw
    )
    
    print(f"\n🔑 Tailles :")
    print(f"   Clé privée  : {len(private_bytes)} bytes (256 bits)")
    print(f"   Clé publique : {len(public_bytes)} bytes (256 bits)")
    print(f"   → Plus compactes que RSA-2048 (256 bytes) !")
    
    # Message
    message = b"Secure message signed with Ed25519"
    print(f"\n📝 Message : {message}")
    
    # Signature
    print(f"\n✍️  Signature Ed25519 :")
    start = time.perf_counter()
    signature = private_key.sign(message)
    time_sign = time.perf_counter() - start
    
    print(f"   Temps : {time_sign*1000:.2f} ms")
    print(f"   Signature (hex) : {signature.hex()}")
    print(f"   Taille : {len(signature)} bytes (512 bits)")
    
    # Vérification
    print(f"\n✅ Vérification :")
    start = time.perf_counter()
    try:
        public_key.verify(signature, message)
        time_verify = time.perf_counter() - start
        print(f"   Temps : {time_verify*1000:.2f} ms")
        print(f"   Résultat : VALIDE ✅")
    except Exception as e:
        print(f"   Résultat : INVALIDE ❌")
        print(f"   Erreur : {e}")
    
    return private_key, public_key, signature

ed25519_private, ed25519_public, ed25519_sig = ed25519_demo()

## 7. Comparaison : RSA vs ECDSA vs Ed25519

In [ ]:
import pandas as pd

def compare_signature_schemes():
    """
    Compare RSA, ECDSA, et Ed25519.
    """
    print("\n" + "=" * 70)
    print("COMPARAISON : RSA vs ECDSA vs Ed25519")
    print("=" * 70)
    
    comparison = pd.DataFrame({
        'Propriété': [
            'Basé sur',
            'Taille clé publique',
            'Taille clé privée',
            'Taille signature',
            'Sécurité (bits)',
            'Vitesse signature',
            'Vitesse vérification',
            'Nonce management',
            'Side-channel resistant',
            'Standardisation'
        ],
        'RSA-2048': [
            'Factorisation',
            '256 bytes',
            '256 bytes',
            '256 bytes',
            '112 bits',
            'Lent',
            'Rapide',
            'N/A',
            'Non',
            'PKCS#1, X.509'
        ],
        'ECDSA P-256': [
            'ECDLP',
            '64 bytes',
            '32 bytes',
            '~64 bytes',
            '128 bits',
            'Rapide',
            'Rapide',
            'Aléatoire (risqué)',
            'Non',
            'FIPS 186-4'
        ],
        'Ed25519': [
            'ECDLP',
            '32 bytes',
            '32 bytes',
            '64 bytes',
            '128 bits',
            'Très rapide',
            'Très rapide',
            'Déterministe (sûr)',
            'Oui',
            'RFC 8032'
        ]
    })
    
    print()
    print(comparison.to_string(index=False))
    
    print(f"\n📊 Benchmark (signatures sur 100 messages) :")
    
    message = b"Test message"
    iterations = 100
    
    # Ed25519
    start = time.perf_counter()
    for _ in range(iterations):
        sig = ed25519_private.sign(message)
        ed25519_public.verify(sig, message)
    time_ed25519 = time.perf_counter() - start
    
    print(f"\n   Ed25519      : {time_ed25519:.3f}s ({time_ed25519/iterations*1000:.2f} ms/op)")
    print(f"   ECDSA P-256  : ~{time_ed25519*1.5:.3f}s (estimé, ~1.5x plus lent)")
    print(f"   RSA-2048     : ~{time_ed25519*10:.3f}s (estimé, ~10x plus lent)")
    
    print(f"\n✅ RECOMMANDATIONS :")
    print(f"   Ed25519   : Nouveau code (SSH, Signal, Tor)")
    print(f"   ECDSA     : Compatibilité (Bitcoin, TLS legacy)")
    print(f"   RSA       : Legacy (TLS 1.2, PGP, X.509 existants)")

compare_signature_schemes()

## 8. Applications Réelles

In [ ]:
print("\n" + "=" * 70)
print("APPLICATIONS RÉELLES")
print("=" * 70)

print(f"\n🌐 TLS/HTTPS :")
print(f"   - TLS 1.2 : RSA-2048 ou ECDSA P-256/P-384")
print(f"   - TLS 1.3 : Préfère ECDSA (plus rapide handshake)")
print(f"   - Certificats X.509 : RSA ou ECDSA")

print(f"\n🔐 SSH :")
print(f"   - ssh-rsa : RSA-2048 (legacy)")
print(f"   - ecdsa-sha2-nistp256 : ECDSA P-256")
print(f"   - ssh-ed25519 : Ed25519 (recommandé) ✅")
print(f"   Commande : ssh-keygen -t ed25519")

print(f"\n💰 Cryptomonnaies :")
print(f"   - Bitcoin : ECDSA secp256k1")
print(f"   - Ethereum : ECDSA secp256k1 (récupération clé publique)")
print(f"   - Monero : Ed25519 (ring signatures)")
print(f"   - Cardano : Ed25519")

print(f"\n📱 Messagerie :")
print(f"   - Signal Protocol : X25519 (ECDH) + Ed25519 (signatures)")
print(f"   - WhatsApp : Signal Protocol")
print(f"   - iMessage : ECDSA (Apple)")

print(f"\n🔏 Autres :")
print(f"   - PGP/GPG : RSA-2048/4096 ou ECDSA")
print(f"   - DNSSEC : RSA ou ECDSA P-256")
print(f"   - Code signing : RSA ou ECDSA (Apple, Microsoft)")
print(f"   - Tor : Ed25519 (onion addresses v3)")

## Conclusion

**Points clés** :
- Courbes elliptiques : Clés courtes (256 bits) = Sécurité RSA-3072
- ✅ ECDSA : Standard NIST, largement déployé
- ⚠️ ECDSA : Vulnérable si nonce k réutilisé ou faible
- ✅ Ed25519 : Courbe moderne, rapide, résistant aux side-channels
- ✅ Ed25519 : Nonce déterministe (pas de risque de réutilisation)

**Sécurité** :
- ECDSA P-256 / Ed25519 : ~128 bits (équivalent RSA-3072)
- ECDSA P-384 : ~192 bits (équivalent RSA-7680)
- ECDSA P-521 : ~256 bits (équivalent RSA-15360)

**Performance** :
- Ed25519 : ~10x plus rapide que RSA-2048
- ECDSA : ~5-8x plus rapide que RSA-2048
- Clés et signatures beaucoup plus compactes

**En pratique** :
- Nouveau code : **Ed25519** (recommandé)
- Compatibilité : **ECDSA P-256**
- Legacy : **RSA-2048** minimum (préférer 3072)

**Éviter** :
- ❌ RSA < 2048 bits
- ❌ ECDSA avec nonce non-aléatoire
- ❌ ECDSA sans RFC 6979 (nonce déterministe)